# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={ODBC Driver 17 for SQL Server};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "UID=WOODFIELD\\jmeza@wdsrx.com;"
                        "PWD=Clavezaza2022;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Feb 13 2023 10:40AM,257233,10,0086372245,ISDIN Corporation,Released
1,Feb 13 2023 10:40AM,257233,10,0086372247,ISDIN Corporation,Released
2,Feb 13 2023 10:40AM,257233,10,0086372251,ISDIN Corporation,Released
3,Feb 13 2023 10:40AM,257233,10,0086372255,ISDIN Corporation,Released
4,Feb 13 2023 10:40AM,257233,10,0086372253,ISDIN Corporation,Released
5,Feb 13 2023 10:40AM,257233,10,0086372257,ISDIN Corporation,Released
6,Feb 13 2023 10:40AM,257233,10,0086372260,ISDIN Corporation,Released
7,Feb 13 2023 10:40AM,257233,10,0086372259,ISDIN Corporation,Released
8,Feb 13 2023 10:40AM,257233,10,0086372265,ISDIN Corporation,Released
9,Feb 13 2023 10:40AM,257233,10,0086372261,ISDIN Corporation,Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
33,257230,Released,1
34,257233,Released,46
35,257234,Released,2
36,257235,Released,2
37,257236,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Executing,Released
id,,
257230,NaN,1.0
257233,NaN,46.0
257234,NaN,2.0
257235,NaN,2.0
257236,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Executing,Released
id,,
257179,0.0,1.0
257186,1.0,2.0
257191,10.0,2.0
257194,40.0,1.0
257195,0.0,42.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Executing,Released
id,,
257179,0,1
257186,1,2
257191,10,2
257194,40,1
257195,0,42


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Executing,Released
0,257179,0,1
1,257186,1,2
2,257191,10,2
3,257194,40,1
4,257195,0,42


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Executing,Released
0,257179,,1
1,257186,1,2
2,257191,10,2
3,257194,40,1
4,257195,,42


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation
46,Feb 13 2023 10:39AM,257236,10,ISDIN Corporation
47,Feb 13 2023 10:38AM,257235,10,Emerginnova
49,Feb 13 2023 10:37AM,257234,10,"Snap Medical Industries, LLC"
51,Feb 13 2023 10:31AM,257230,22,"NBTY Global, Inc."
52,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated"
53,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc."
57,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation
119,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation
169,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Executing,Released
0,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation,,46
1,Feb 13 2023 10:39AM,257236,10,ISDIN Corporation,,1
2,Feb 13 2023 10:38AM,257235,10,Emerginnova,,2
3,Feb 13 2023 10:37AM,257234,10,"Snap Medical Industries, LLC",,2
4,Feb 13 2023 10:31AM,257230,22,"NBTY Global, Inc.",,1
5,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated",,1
6,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc.",,4
7,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation,,62
8,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation,,50
9,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc.",,6


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation,46,
1,Feb 13 2023 10:39AM,257236,10,ISDIN Corporation,1,
2,Feb 13 2023 10:38AM,257235,10,Emerginnova,2,
3,Feb 13 2023 10:37AM,257234,10,"Snap Medical Industries, LLC",2,
4,Feb 13 2023 10:31AM,257230,22,"NBTY Global, Inc.",1,
5,Feb 13 2023 10:29AM,257229,10,"Senseonics, Incorporated",1,
6,Feb 13 2023 10:27AM,257228,15,"Alliance Pharma, Inc.",4,
7,Feb 13 2023 10:27AM,257219,10,ISDIN Corporation,62,
8,Feb 13 2023 10:26AM,257217,10,ISDIN Corporation,50,
9,Feb 13 2023 10:26AM,257227,15,"Alliance Pharma, Inc.",6,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation,46,
1,Feb 13 2023 10:39AM,257236,10,ISDIN Corporation,1,
2,Feb 13 2023 10:38AM,257235,10,Emerginnova,2,
3,Feb 13 2023 10:37AM,257234,10,"Snap Medical Industries, LLC",2,
4,Feb 13 2023 10:31AM,257230,22,"NBTY Global, Inc.",1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation,46.0,NaN
1,Feb 13 2023 10:39AM,257236,10,ISDIN Corporation,1.0,NaN
2,Feb 13 2023 10:38AM,257235,10,Emerginnova,2.0,NaN
3,Feb 13 2023 10:37AM,257234,10,"Snap Medical Industries, LLC",2.0,NaN
4,Feb 13 2023 10:31AM,257230,22,"NBTY Global, Inc.",1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Feb 13 2023 10:40AM,257233,10,ISDIN Corporation,46.0,0.0
1,Feb 13 2023 10:39AM,257236,10,ISDIN Corporation,1.0,0.0
2,Feb 13 2023 10:38AM,257235,10,Emerginnova,2.0,0.0
3,Feb 13 2023 10:37AM,257234,10,"Snap Medical Industries, LLC",2.0,0.0
4,Feb 13 2023 10:31AM,257230,22,"NBTY Global, Inc.",1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,5658685,436.0,1.0
12,514423,3.0,0.0
15,771677,95.0,8.0
19,1028778,45.0,51.0
21,514441,2.0,0.0
22,257230,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,5658685,436.0,1.0
1,12,514423,3.0,0.0
2,15,771677,95.0,8.0
3,19,1028778,45.0,51.0
4,21,514441,2.0,0.0
5,22,257230,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,436.0,1.0
1,12,3.0,0.0
2,15,95.0,8.0
3,19,45.0,51.0
4,21,2.0,0.0
5,22,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,436.0
1,12,Released,3.0
2,15,Released,95.0
3,19,Released,45.0
4,21,Released,2.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,15,19,21,22
Status,,,,,,
Executing,1.0,0.0,8.0,51.0,0.0,0.0
Released,436.0,3.0,95.0,45.0,2.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,15,19,21,22
0,Executing,1.0,0.0,8.0,51.0,0.0,0.0
1,Released,436.0,3.0,95.0,45.0,2.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,15,19,21,22
0,Executing,1.0,0.0,8.0,51.0,0.0,0.0
1,Released,436.0,3.0,95.0,45.0,2.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()